<img src="images/reg1.png" style="height: 400px">

In [1]:
from tensorflow.keras.applications.resnet50 import preprocess_input 
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
import numpy as np
import cv2
import cv2.ximgproc.segmentation as segmentation

# birden fazla tespit edilen objeyi azaltmak için
%run ../3_non_max_supression/3.ipynb 

## 1-Create Selective Search Method

In [2]:
def selective_search(image):
    print("ss")

    # initiliaze selective search segmantation algorithm
    ss = segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchQuality()
    
    print("Segmentation started...")
    rects = ss.process()
    print("\nSegmentation has completed succesfully...")

    print("Length of segmentations:", len(rects))
    
    return rects[:1000] # first 1000 rectangles

## 2-Applying RestNet50 Model

In [3]:
model = ResNet50(weights="imagenet")    
image = cv2.imread("images/animals.jpg")
image = cv2.resize(image, dsize = (400,400))
(H, W) = image.shape[:2]

## 3-Applying Selective Search

In [4]:
rects = selective_search(image)

ss
Segmentation started...

Segmentation has completed succesfully...
Length of segmentations: 5865


## 4-Find Region of Interest (ROI)

In [5]:
proposals = []
boxes = []
for (x, y, w, h) in rects:

    # width and height, < region/10 => continue
    if w / float(W) < 0.1 or h / float(H) < 0.1: continue
    
    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224, 224))

    roi = img_to_array(roi)
    roi = preprocess_input(roi)

    proposals.append(roi)
    boxes.append((x, y, w, h))

proposals = np.array(proposals)

## 5-Predict Image

In [6]:
preds = model.predict(proposals)
preds = imagenet_utils.decode_predictions(preds, top=1)
preds[0:10]

[[('n01806143', 'peacock', 0.9970944)],
 [('n02510455', 'giant_panda', 0.8123711)],
 [('n02510455', 'giant_panda', 0.9812214)],
 [('n02115913', 'dhole', 0.13086034)],
 [('n02423022', 'gazelle', 0.68198144)],
 [('n02486261', 'patas', 0.62203395)],
 [('n02356798', 'fox_squirrel', 0.9906234)],
 [('n02129165', 'lion', 0.4946193)],
 [('n01608432', 'kite', 0.17909473)],
 [('n02510455', 'giant_panda', 0.96424514)]]

In [7]:
labels = {}
min_conf = 0.8 # select data with probability above 0.8
for (i, p) in enumerate(preds):
    
    
    (_, label, prob) = p[0]
    if prob >= min_conf:
        (x, y, w, h) = boxes[i]
        box = (x, y, x + w, y + h)
        L = labels.get(label, [])
        L.append((box, prob))
        labels[label] = L

## 6-Appliying Non Max Suppression

In [8]:
clone = image.copy()

for label in labels.keys():
    for (box, prob) in labels[label]:
        boxes = np.array([p[0] for p in labels[label]])
        proba = np.array([p[1] for p in labels[label]])
        boxes = non_max_suppression(boxes, proba)
    
        for (startX, startY, endX, endY) in boxes:
            cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 0, 255), 2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(clone, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
cv2.imshow("After", clone)
if cv2.waitKey(0) == ord("q"): 
    cv2.destroyAllWindows()